In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Agent Engine in Express Mode

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb">
      <img width="32px" src="https://storage.googleapis.com/github-repo/generative-ai/logos/GitHub_Invertocat_Dark.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/agent_engine_express_mode.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

## Set up the environment

https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#sdk-installation

In [ ]:
%pip install "google-cloud-aiplatform[agent_engines]" --force-reinstall --quiet

# Sign up for Vertex AI Express mode

We can use Vertex Services like Agent Engine and access models without requiring a credit card through Vertex AI Express Mode!

Sign up with your google account here: https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview to unlock access to certain services and the latest google models models!

Vertex Express Mode combined with ADK allow for the creation of advanced agents with customizable tools and a context layer with sessions and memories without any billing!

Once you sign up, paste your api key here:

In [ ]:
express_mode_api_key = ""  # @param {type:"string"}

# Develop an Adk agent locally

### Step 1: Define a local simple agent

We will use AdkApp, which is a template to make Adk Agents to create a simple agent that has no tools, just a model. We will be able to converse with it, but it cannot help us with the currency conversion rates.

With Vertex AI Express mode, you have access to the latest Google models. For the comprehensive list, see the [Available models and rate limits in express mode](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview#models). We will be using Gemini 2.5 Flash for this example.

In [ ]:
# Initialize Vertex AI SDK
import vertexai
from vertexai import agent_engines

vertexai.init(api_key=express_mode_api_key)

In [ ]:
model = "gemini-2.5-flash"

In [ ]:
from google.adk.agents import Agent
from google.adk.apps import App

agent = Agent(
    model=model,  # Required.
    name="currency_exchange_agent",  # Required.
)

In [ ]:
app = agent_engines.AdkApp(
    app=App(
        name="currency_exchange_app",
        root_agent=agent,
    )
)

In [ ]:
async for event in app.async_stream_query(
    user_id="local user",  # Required
    message="What is the exchange rate from US dollars to Swedish krona on 2025-09-25?",
):
    print(event)

### Step 2: Define a tool

Now that we know we can talk to the agent, lets add a tool. This tool allows the agent to check the exchange rate between two currencies ona given date using the frankfurter app.

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

To test the function before you use it in your agent, run the following:

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

### Step 3: Add the tool to your agent and test locally

You have to associate the tools with the Agent when it is initialized:

In [ ]:
from google.adk.agents import Agent

agent = Agent(
    model=model,  # Required.
    name="currency_exchange_agent",  # Required.
    tools=[get_exchange_rate],  # [new] Optional.
)

Then, we test locally it like we did before.

In [ ]:
app = agent_engines.AdkApp(
    app=App(
        name="currency_exchange_app",
        root_agent=agent,
    )
)
async for event in app.async_stream_query(
    user_id="local user",
    message="What is the exchange rate from US dollars to Swedish krona on 2025-09-25?",
):
    print(event)

# Deploy your Agent with Agent Engine

### Step 4: Write agent files

Now that we know our agent will work locally, we can write our code to the local files and deploy it to Google Cloud without cost to Agent Engine with source based deployment! More info can be found at [Deploy an Agent Engine](https://docs.cloud.google.com/agent-builder/agent-engine/deploy#from-source-files).

In [ ]:
!mkdir test

In [ ]:
%%writefile test/requirements.txt
google-cloud-aiplatform[agent_engines,adk]

In [ ]:
%%writefile test/my_agent.py
from google.adk.agents import Agent
from vertexai import agent_engines

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

agent = Agent(
    model="gemini-2.5-flash",   # Required.
    name='currency_exchange_agent',  # Required.
    tools=[get_exchange_rate],       # [new] Optional.
)

adk_app=agent_engines.AdkApp(agent=agent, enable_tracing=True)

In [ ]:
# Define Class Methods
class_methods = [
    {
        "parameters": {},
        "api_mode": "async_stream",
        "description": None,
        "name": "async_stream_query",
    },
    {
        "parameters": {},
        "api_mode": "async",
        "description": None,
        "name": "async_create_session",
    },
]

### Step 5: Deploy your agent to Agent Engine

Deploying your agent to Agent Engine unlocks many Google Cloud services, including the Vertex AI Memory Bank and Session services and a suite of observability, logging, and tracing. All of the conversations with the agent are stored in Vertex AI and can be used to give the agent context for future conversations through long term memories. Visit the Vertex AI documentation to learn more about the [Vertex AI Session service](https://docs.cloud.google.com/agent-builder/agent-engine/sessions/overview) and the [Vertex AI Memory Bank service](https://docs.cloud.google.com/agent-builder/agent-engine/memory-bank/overview).

In [ ]:
client = vertexai.Client(api_key=express_mode_api_key)

remote_agent = client.agent_engines.create(
    config={
        "display_name": "test agent",
        "description": "test agent",
        "source_packages": ["test"],
        "entrypoint_module": "test.my_agent",
        "entrypoint_object": "adk_app",
        "requirements_file": "test/requirements.txt",
        "class_methods": class_methods,
        "agent_framework": "google-adk",
    },
)
remote_agent

### Step 6: Talk to your deployed Agent Engine

In [ ]:
async for item in remote_agent.async_stream_query(
    message="What is the exchange rate between USD and Korean Won on 2025-09-25?",
    user_id="user1",
):
    print(item)

In [ ]:
async for item in remote_agent.async_stream_query(
    message="What can you help me with?",
    user_id="user1",
):
    print(item)

# Manage your Agent Engine deployment

### Step 7: Talk with your agent through the Agent Engine UI

You can also converse with your deployed agent though the Agent Engine UI in your express mode console. Visit the [Express mode UI](https://console.cloud.google.com/expressmode) and navigate to the Agent Engine menu.

![agent-engine-ui](https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/express-mode/agent-engine-ui.png)

Once you select your agent, use the Playground to converse with it!

![playground](https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/express-mode/playground.png)

### Step 8: View conversation history with Vertex AI Session Service

In [ ]:
session = list(client.agent_engines.sessions.list(name=remote_agent.api_resource.name))[
    0
]

In [ ]:
list(client.agent_engines.sessions.events.list(name=session.name))